In [ ]:
pip install langchain langchain_openai langchain_community chromadb

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
# 쿼리를 위한 로그 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharecterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("문서 경로")]

docs = []
for loader in loaders:
  docs.extend(loader.load())

In [ ]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings()

# Chroma vectorstore 설정
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

In [ ]:
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 설정(여기서는 ChatOpenAI 사용)
llm = ChatOpenAi(model = "gpt-4o", temperature=0.2)

# MultiQueryRetriever 실행
retriever = MultiQueryRetriever.from_llm(
    # 기본 검색기(벡터 데이터베이스)
    retriever=vectorstore.as_retriever(),
    # 앞서 정의한 llm(gpt-4o)
    llm=llm
)

In [ ]:
# 예시 질문
question = "주식 투자를 처음 시작하려면 어떻게 해야 하나요?"

# 결과 검색
unique_docs = retriever.invoke(question)
print(f"\n결과: {len(unique_docs)}개의 문서가 검색되었습니다.")

In [ ]:
from langchain.chains import RetrivalQA

# RetrievalQA 체인 설정
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 질문에 대한 답변 생성
result =  qa_chain.invoke({"query": question})

# 결과 출력
print("답변:", result["result"])
print("\n사용된 문서:")

for doc in result["source_documents"]:
  print(doc.page_content)